In [3]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random

from auxiliaries import *

random.seed(34)

In [23]:
# types of routes
types_helsinki = {
    0: "tram",
    1: "subway",
    2: "rail",
    3: "bus",
    4: "ferry",
    5: "walking",
}

In [38]:
# STEP 1
# read in public transport network of helsinki and walking network
helsinki, helsinki_nodes = read_in_network("helsinki", "combined")
helsinki_walking, nodes_t = read_in_network("helsinki", "walk")

# adapt helsinki_walking dataframe to match helsinki dataframe
helsinki_walking = helsinki_walking.rename(columns={"d_walk": "duration_avg"})
# filter out edges that are not in helsinki_nodes
# make list of nodes in helsinki from helsinki_nodes
nodes_h = list(helsinki["from_stop_I"].unique())
helsinki_walking = helsinki_walking[helsinki_walking["from_stop_I"].isin(nodes_h)]
helsinki_walking = helsinki_walking[helsinki_walking["to_stop_I"].isin(nodes_h)]

helsinki_walking["duration"] = helsinki_walking["duration_avg"].apply(distance_to_duration)
helsinki_walking["n_vehicles"] = 0
helsinki_walking["route_I_counts"] = 0
helsinki_walking["route_type"] = 5

# add helsinki_walking to helsinki
helsinki_full = pd.concat([helsinki, helsinki_walking], ignore_index=True)
helsinki_walking

from_stop_I  to_stop_I    d  duration_avg    duration  n_vehicles   
1                 1          2  146           155  119.230769           0  \
2                 1          3  325           357  274.615385           0   
3                 1          4  337           350  269.230769           0   
4                 1          5  406           567  436.153846           0   
5                 1          6  397           550  423.076923           0   
...             ...        ...  ...           ...         ...         ...   
101516         7522       7524  433           479  368.461538           0   
101517         7522       7525  474           493  379.230769           0   
101518         7523       7524  348           398  306.153846           0   
101519         7523       7525  389           413  317.692308           0   
101520         7524       7525   42            61   46.923077           0   

        route_I_counts  route_type  
1                    0           5  
2                    0           5  
3                    0           5  
4                    0           5  
5                    0           5  
...                ...         ...  
101516               0           5  
101517               0           5  
101518               0           5  
101519               0           5  
101520               0           5  

[95840 rows x 8 columns]

In [39]:
# STEP 2
# convert to graph
G = convert_to_graph(helsinki_full)
pos = add_positions(G, helsinki_nodes)

# STEP 3
# convert walking network to seperate graph
G_walk = convert_to_graph(helsinki_walking)

In [40]:
len(G_walk.nodes())

6891

In [41]:
# make G connected by adding walking edges
make_walking_connected(G, G_walk)

In [ ]:
# STEP 4
results = percolation_experiments(G, 0, 1, 20, 1, 5, verbose=True)